In [5]:
import pandas as pd
from underthesea import word_tokenize
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
from numpy import hstack
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import hstack

In [6]:
dh = pd.read_csv("Dataset_articles_NoID.csv")
df = dh.head(1000)
df

,URL,Title,Summary,Contents,Date,Author(s),Category,Tags
0,https://laodong.vn/bat-dong-san/thong-tin-ngoc...,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,https://laodong.vn/bat-dong-san/lo-hong-trong-...,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản,"['Thủ Thiêm', 'Đấu giá đất']"
2,https://laodong.vn/bat-dong-san/som-hoan-thien...,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,https://laodong.vn/bat-dong-san/chi-tiet-ho-so...,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,https://laodong.vn/bat-dong-san/khoi-tao-khong...,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản,['An Quý Villa']
...,...,...,...,...,...,...,...,...
995,https://laodong.vn/bat-dong-san/can-cu-giai-qu...,Căn cứ giải quyết tranh chấp về lối đi khi khô...,Dưới đây là căn cứ giải quyết tranh chấp về lố...,Căn cứ giải quyết tranh chấp về lối đi khi khô...,"Thứ bảy, 27/11/2021 08:30 (GMT+7)",Trang Thiều (T/H),Bất động sản,"['Giấy chứng nhận quyền sử dụng đất', 'Sổ đỏ',..."
996,https://laodong.vn/bat-dong-san/thanh-hoa-don-...,Thanh Hóa đón dòng sản phẩm mới Shop Luxury Eu...,Eurowindow Holding - nhà phát triển dự án Euro...,Shop thương mại lâu nay vẫn là “miếng bánh ngo...,"Thứ sáu, 26/11/2021 19:00 (GMT+7)",H.Ly,Bất động sản,"['Eurowindow Holding', 'Eurowindow', 'Eurowind..."
997,https://laodong.vn/bat-dong-san/bat-dong-san-y...,Bất động sản Yên Bái nhộn nhịp đón nhà đầu tư,Nhờ chiến lược tập trung phát triển đô thị bài...,"Những năm qua, Yên Bái đã là một trong những t...","Thứ sáu, 26/11/2021 19:00 (GMT+7)",H.Ly,Bất động sản,"['Eurowindow Holding', 'Eurowindow', 'Melinh P..."
998,https://laodong.vn/bat-dong-san/tphcm-khong-ta...,TPHCM không tăng giá đất năm 2022 vì dịch COVI...,TPHCM thống nhất giữ nguyên hệ số điều chỉnh g...,Văn phòng UBND TPHCM vừa thông báo kết luận củ...,"Thứ sáu, 26/11/2021 16:37 (GMT+7)",MINH QUÂN,Bất động sản,"['Bảng giá đất TPHCM', 'Dịch COVID-19']"


In [7]:
#Loại bỏ hàng NULL/NA
df = df.dropna(axis=0, how='any')
df

,URL,Title,Summary,Contents,Date,Author(s),Category,Tags
0,https://laodong.vn/bat-dong-san/thong-tin-ngoc...,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,https://laodong.vn/bat-dong-san/lo-hong-trong-...,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản,"['Thủ Thiêm', 'Đấu giá đất']"
2,https://laodong.vn/bat-dong-san/som-hoan-thien...,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,https://laodong.vn/bat-dong-san/chi-tiet-ho-so...,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,https://laodong.vn/bat-dong-san/khoi-tao-khong...,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản,['An Quý Villa']
...,...,...,...,...,...,...,...,...
995,https://laodong.vn/bat-dong-san/can-cu-giai-qu...,Căn cứ giải quyết tranh chấp về lối đi khi khô...,Dưới đây là căn cứ giải quyết tranh chấp về lố...,Căn cứ giải quyết tranh chấp về lối đi khi khô...,"Thứ bảy, 27/11/2021 08:30 (GMT+7)",Trang Thiều (T/H),Bất động sản,"['Giấy chứng nhận quyền sử dụng đất', 'Sổ đỏ',..."
996,https://laodong.vn/bat-dong-san/thanh-hoa-don-...,Thanh Hóa đón dòng sản phẩm mới Shop Luxury Eu...,Eurowindow Holding - nhà phát triển dự án Euro...,Shop thương mại lâu nay vẫn là “miếng bánh ngo...,"Thứ sáu, 26/11/2021 19:00 (GMT+7)",H.Ly,Bất động sản,"['Eurowindow Holding', 'Eurowindow', 'Eurowind..."
997,https://laodong.vn/bat-dong-san/bat-dong-san-y...,Bất động sản Yên Bái nhộn nhịp đón nhà đầu tư,Nhờ chiến lược tập trung phát triển đô thị bài...,"Những năm qua, Yên Bái đã là một trong những t...","Thứ sáu, 26/11/2021 19:00 (GMT+7)",H.Ly,Bất động sản,"['Eurowindow Holding', 'Eurowindow', 'Melinh P..."
998,https://laodong.vn/bat-dong-san/tphcm-khong-ta...,TPHCM không tăng giá đất năm 2022 vì dịch COVI...,TPHCM thống nhất giữ nguyên hệ số điều chỉnh g...,Văn phòng UBND TPHCM vừa thông báo kết luận củ...,"Thứ sáu, 26/11/2021 16:37 (GMT+7)",MINH QUÂN,Bất động sản,"['Bảng giá đất TPHCM', 'Dịch COVID-19']"


In [8]:
#Viết hoa cột Author(s)
df.loc[:, "Author(s)"] = df["Author(s)"].str.upper()
#Bỏ cột URL và Date
df = df.drop("URL", axis=1)
df = df.drop("Date", axis=1)
df = df.drop("Tags", axis=1)

In [9]:
# Tạo một danh sách stopword đơn giản cho tiếng Việt
stop_words = set(["là", "của", "và", "có", "được", "trong", "với", "cho", "đến"])
# Lấy một danh sách stopword cho tiếng Việt từ file
with open("vietnamese-stopwords.txt", "r", encoding="utf-8") as f:
    new_stop_words = set(f.read().splitlines())
    stop_words.update(new_stop_words)

# Định nghĩa hàm chuẩn hóa dữ liệu văn bản
def normalize_text(text):
    # Loại bỏ ký tự đặc biệt và chuyển đổi sang chữ thường
    text = "".join(char for char in text if char.isalnum() or char.isspace()).lower()
    # Tách từ
    words = word_tokenize(text)
    # Loại bỏ stopword
    words = [word for word in words if word not in stop_words]
    # Nối lại các từ đã xử lý
    return " ".join(words)

# Chuẩn hóa dữ liệu văn bản cho cột "Title" và "Contents"
df.loc[:, "Title"] = df["Title"].apply(normalize_text)
df.loc[:, "Summary"] = df["Summary"].apply(normalize_text)
df.loc[:, "Contents"] = df["Contents"].apply(normalize_text)


In [10]:
# # Khởi tạo đối tượng MultiLabelBinarizer
# mlb = MultiLabelBinarizer()

# # Mã hóa cột "Thẻ"
# encoded_tags = mlb.fit_transform(df['Tags'])

In [11]:
# # Khởi tạo đối tượng PCA
# pca = PCA(n_components=10)

# # Giảm kích thước của ma trận đặc trưng cho cột "Tags"
# encoded_tags = pca.fit_transform(encoded_tags)

# # Thêm các cột mới vào DataFrame
# df = df.join(pd.DataFrame(encoded_tags))

# #Bỏ cột Tags
# df = df.drop("Tags", axis=1)

In [12]:
# Đọc dữ liệu
data = df
data

,Title,Summary,Contents,Author(s),Category
0,thông tin ngọc trinh mua đất bảo lộc chiêu trò...,lâm đồng lãnh đạo thành phố bảo lộc lâm đồng t...,trang facebook chủ ngọc trinh đăng tải thông t...,PHƯƠNG NHIÊN,Bất động sản
1,lỗ hổng thẩm tra năng lực tài chính tham gia đ...,tphcm cưỡng chế thuế hai doanh nghiệp trúng đấ...,thông tin cục thuế tphcm hiện cơ quan chức năn...,GIA MIÊU,Bất động sản
2,hoàn thiện dự án nhà ở xã hội cnlđ mua,hiện địa bàn tỉnh ninh bình 32 khu cụm công ng...,cnlđ mong muốn tiếp cận nhà ở xã hội khảo sát ...,NGUYỄN TRƯỜNG,Bất động sản
3,chi tiết hồ sơ hoàn công 2022,hoàn công ý nghĩa đổi sổ đỏ thể hiện hiện trạn...,hoàn công nhà ở thủ tục hành chính hoạt động x...,KIM NHUNG (T/H),Bất động sản
4,khởi không gian sống đẳng cấp đón sóng đầu tư ...,dự án nội đô an quý villa giải khát lý tưởng n...,đi dọc đường lê văn lương kéo dài khu dương nộ...,HUYỀN NGUYỄN,Bất động sản
...,...,...,...,...,...
995,căn cứ giải quyết tranh chấp lối đi sổ đỏ,căn cứ giải quyết tranh chấp lối đi sổ đỏ nghị...,căn cứ giải quyết tranh chấp lối đi sổ đỏ căn ...,TRANG THIỀU (T/H),Bất động sản
996,thanh hóa đón dòng sản phẩm shop luxury eurowi...,eurowindow holding phát triển dự án eurowindow...,shop thương mại miếng bánh ngon giới đầu tư bấ...,H.LY,Bất động sản
997,bất động sản yên bái nhộn nhịp đón nhà đầu tư,chiến lược phát triển đô thị bài bản tỉnh yên ...,yên bái tỉnh phát triển năng động khu vực tốc ...,H.LY,Bất động sản
998,tphcm tăng giá đất 2022 dịch covid19,tphcm thống nhất nguyên hệ số điều chỉnh giá đ...,văn phòng ubnd tphcm thông báo kết luận phó ch...,MINH QUÂN,Bất động sản


In [13]:
# Khởi tạo vectorizer
vectorizer = TfidfVectorizer()

# Tính toán TF-IDF cho tất cả các thuộc tính
vectorizer.fit(data['Title'] + data['Summary'] + data['Contents'])
title_tfidf = vectorizer.transform(data['Title'])
summary_tfidf = vectorizer.transform(data['Summary'])
contents_tfidf = vectorizer.transform(data['Contents'])

# Kết hợp các ma trận TF-IDF lại với nhau
# (có thể sử dụng các phương pháp khác nhau để kết hợp)
features = hstack([title_tfidf, summary_tfidf, contents_tfidf])

In [14]:
# Khởi tạo encoder
encoder = OneHotEncoder()

# Mã hóa các thuộc tính phân loại
category_encoded = encoder.fit_transform(data[['Category']])
author_encoded = encoder.fit_transform(data[['Author(s)']])

# Kết hợp các ma trận lại với nhau
features = hstack([features, category_encoded])
features = hstack([features, author_encoded])

In [15]:
print(features)

  (0, 8241)	0.22705413090191615
  (0, 7064)	0.3147046016680084
  (0, 6999)	0.5050845042689536
  (0, 6856)	0.17969818326105758
  (0, 6821)	0.3262745129935247
  (0, 6729)	0.12635396342566166
  (0, 5333)	0.32094079071750714
  (0, 4947)	0.16231094490089681
  (0, 4795)	0.3276608726121716
  (0, 3536)	0.15068922751843836
  (0, 2601)	0.36563374617533223
  (0, 2435)	0.21559336458388464
  (0, 17104)	0.07653049305047521
  (0, 17031)	0.15894868633596865
  (0, 16967)	0.08618472984598645
  (0, 16838)	0.14157810861408035
  (0, 16825)	0.12790918456063702
  (0, 15661)	0.1962319826524159
  (0, 15596)	0.3149421176378842
  (0, 15453)	0.11204961920942624
  (0, 15418)	0.20344632469408014
  (0, 15383)	0.2602880408468984
  (0, 15326)	0.07878718209899299
  (0, 15292)	0.07555700165477432
  (0, 15151)	0.15045475786759033
  :	:
  (999, 20387)	0.03808560988169731
  (999, 20358)	0.11523672424268781
  (999, 20319)	0.08487027610969379
  (999, 20175)	0.04384149325859618
  (999, 20145)	0.10082204595999818
  (999, 20125

In [16]:
#Features là ma trận đặc trưng đã tính toán ở bước trước

# Khởi tạo mô hình
model = NearestNeighbors()

# Huấn luyện mô hình
model.fit(features)

# Dự đoán mức độ tương tự giữa các bài viết
# (ví dụ: tìm 5 bài viết tương tự nhất với bài viết đầu tiên)
distances, indices = model.kneighbors(features[0], n_neighbors=5)

In [25]:
# Dự đoán mức độ tương tự giữa các bài viết
# (ví dụ: tìm 5 bài viết tương tự nhất với bài viết đầu tiên)
distances, indices = model.kneighbors(features, n_neighbors=5)

In [26]:
print("Indices:", indices)
print("Distances:", distances)

Indices: [[  0 870 714 926 630]
 [  1 169  15 474  11]
 [  2 232 457 613 720]
 ...
 [997 981 894 852 939]
 [998 654 872 430 460]
 [999 238 526 179  27]]
Distances: [[0.         2.50114143 2.50691458 2.52563351 2.53770446]
 [0.         1.66972766 1.80858843 1.84599373 1.89706775]
 [0.         2.31267787 2.60308221 2.61416482 2.62705033]
 ...
 [0.         1.63265842 1.88854038 2.23449971 2.28032218]
 [0.         2.17537494 2.2095147  2.25300058 2.31637821]
 [0.         2.20354786 2.40151503 2.51977368 2.52396181]]


In [32]:
split_indices = np.split(indices, indices.shape[1], axis=1)

# Tạo DataFrame từ các mảng con
data = {f'index_{i}': split_indices[i].ravel() for i in range(len(split_indices))}
df = pd.DataFrame(data)

# Xuất DataFrame ra tệp CSV
df.to_csv('ket_qua.csv', index=False, header=False)

In [ ]:
##END